<a href="https://colab.research.google.com/github/drbilal216/Tensorflow-Datasets-Q3/blob/master/Hyperparameter_Optimization_Deep_Learning(Regression)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hyperparameter Tuning of Keras Deep Learning Model in Python

https://mc.ai/hyperparameter-tuning-of-keras-deep-learning-model-in-python/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Data from here
https://drive.google.com/drive/folders/1gaTtFxrqHY0vBzaHAdcJDghcdhaEimiX?usp=sharing

In [ ]:
import pandas as pd
Input_path = '/content/drive/My Drive/ML and AI/Datasets/compresive_strength_concrete/compresive_strength_concrete.csv'
data_df =  pd.read_csv(Input_path)
display(data_df.head())

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [ ]:
# Slicing Features and Labels
Samples = data_df.iloc[:,:-1]
Labels = data_df.iloc[:,-1:]
Samples = Samples.values
Labels = Labels.values
Labels = Labels.ravel()
print()
print(Labels)


[79.99 61.89 40.27 ... 23.7  32.77 32.4 ]


In [ ]:
# Normalizing
mean = Samples.mean()
std = Samples.std()
Samples -= mean
Samples /= std
display(Samples)

array([[ 0.68491213, -0.84750875, -0.84750875, ...,  2.10382035,
         1.07085517, -0.76804989],
       [ 0.68491213, -0.84750875, -0.84750875, ...,  2.1463876 ,
         1.07085517, -0.76804989],
       [ 0.09606522, -0.44311991, -0.84750875, ...,  1.79733617,
         0.83815422, -0.08129831],
       ...,
       [-0.42609301, -0.45191714, -0.53932188, ...,  1.68495864,
         1.36598808, -0.76804989],
       [-0.39601215, -0.31768842, -0.84750875, ...,  1.9607944 ,
         1.39124464, -0.76804989],
       [-0.10712244, -0.5623082 , -0.62530772, ...,  1.60578356,
         1.31348847, -0.76804989]])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Samples, Labels, test_size=0.3, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.285)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

(515, 8)
(515,)
(309, 8)
(309,)
(206, 8)
(206,)


In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

8


In [ ]:
###############################

In [ ]:
# normally we make Sequential model like this
from keras.models import Sequential
from keras import regularizers #for l2 regularization
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

activation = "relu"
learning_rate = 0.01
opt = Adam(lr=learning_rate)

model = Sequential()
model.add(Dense(128,activation = activation,input_shape = (8,)))#,activity_regularizer = regularizers.l2(1e-5)))
model.add(Dropout(0.50))
model.add(Dense(128,activation = activation,activity_regularizer = regularizers.l2(1e-5)))
model.add(Dropout(0.50))
model.add(Dense(1, activation = activation))# Compile the model

model.compile(optimizer = opt,loss = "mean_absolute_error",metrics=['mse', "mape"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1152      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 17,793
Trainable params: 17,793
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train,epochs=2,validation_data=(x_val,y_val))

Epoch 1/2
17/17 [==============================] - 0s 12ms/step - loss: 21.7619 - mse: 735.1840 - mape: 74.5528 - val_loss: 15.2237 - val_mse: 352.1429 - val_mape: 57.9518
Epoch 2/2
17/17 [==============================] - 0s 4ms/step - loss: 15.6840 - mse: 393.9334 - mape: 60.3261 - val_loss: 14.1004 - val_mse: 294.9263 - val_mape: 60.5009


In [ ]:
#####################

### Hyperperameter optimizatoin
Lets try it now

In [ ]:
n_cols = x_train.shape[1]
input_shape = (n_cols, )# input shape

from keras.models import Sequential
from keras import regularizers #for l2 regularization
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

def create_model(learning_rate = 0.01, activation = 'relu'):
 # Create an Adam optimizer with the given learning rate
 opt = Adam(lr=learning_rate)
 # Create your binary classification model 
 model = Sequential()
 model.add(Dense(128,activation = activation,input_shape = input_shape,activity_regularizer = regularizers.l2(1e-5)))
 model.add(Dropout(0.50))
 model.add(Dense(128,activation = activation,activity_regularizer = regularizers.l2(1e-5)))
 model.add(Dropout(0.50))
 model.add(Dense(1, activation = activation))
 # Compile the model
 model.compile(optimizer = opt,loss = "mean_absolute_error",metrics=['mse', "mape"])
 return model

In [ ]:
# Create a KerasClassifier object
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
#from sklearn.model_selection import cross_val_score

model = KerasRegressor(build_fn = create_model,verbose = 0)# Define the hyperparameter space
params = {'activation': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [5, 10], # [50, 100]
          'learning_rate': [0.01, 0.001, 0.0001]}# Create a randomize search cv object 

random_search = RandomizedSearchCV(model,param_distributions = params,cv = KFold(10))

In [ ]:
random_search_results = random_search.fit(x_train, y_train)

In [ ]:
print("Best Score: ",random_search_results.best_score_,
 "and Best Params: ",random_search_results.best_params_)

Best Score:  -12.119218921661377 and Best Params:  {'learning_rate': 0.01, 'epochs': 5, 'batch_size': 16, 'activation': 'relu'}


In [ ]:
# we can use the best model above directly of ML

prediction = random_search.predict(x_test)
#prediction

import numpy as np
predict = 41
print("Predicted Value :",prediction[predict])
print("Actual Value :",prediction[predict])

Predicted Value : 35.460598
Actual Value : 35.460598


### Now using those parameters
Making and running new model with same parameter in Deep Learning

Note: (these steps below are not needed because we already have the model adove)

In [ ]:
learning_rate = 0.01
activation = 'relu'
epochs = 50
batch_size = 32 

model = create_model(learning_rate, activation)
#model.summary()

Model: "sequential_307"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_921 (Dense)            (None, 128)               1152      
_________________________________________________________________
dropout_614 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_922 (Dense)            (None, 128)               16512     
_________________________________________________________________
dropout_615 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_923 (Dense)            (None, 1)                 129       
Total params: 17,793
Trainable params: 17,793
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train, epochs=epochs, batch_size = batch_size,
                    validation_data=(x_val,y_val), verbose = 1)

Epoch 1/50
17/17 [==============================] - 0s 11ms/step - loss: 20.7899 - mse: 674.1634 - mape: 74.7627 - val_loss: 16.1468 - val_mse: 401.6671 - val_mape: 56.1916
Epoch 2/50
17/17 [==============================] - 0s 4ms/step - loss: 15.1645 - mse: 349.0859 - mape: 63.4318 - val_loss: 14.5092 - val_mse: 320.8870 - val_mape: 55.5158
Epoch 3/50
17/17 [==============================] - 0s 4ms/step - loss: 14.1700 - mse: 314.9036 - mape: 58.1071 - val_loss: 14.1083 - val_mse: 310.8538 - val_mape: 49.9547
Epoch 4/50
17/17 [==============================] - 0s 4ms/step - loss: 13.8001 - mse: 288.5428 - mape: 56.3754 - val_loss: 12.9969 - val_mse: 248.9715 - val_mape: 54.7970
Epoch 5/50
17/17 [==============================] - 0s 5ms/step - loss: 13.4697 - mse: 283.9972 - mape: 53.8828 - val_loss: 13.6866 - val_mse: 294.5759 - val_mape: 45.8589
Epoch 6/50
17/17 [==============================] - 0s 5ms/step - loss: 13.5122 - mse: 283.4504 - mape: 52.7597 - val_loss: 12.5075 - val_m

### Another great article about above example
https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/

In [ ]:
################################

### Lets try same practice in Functional API

In [ ]:
from keras import Input, layers, Model
from keras.optimizers import Adam

activation = "relu"
learning_rate = 0.01
opt = Adam(lr=learning_rate)

input_tensor = Input(shape=(8,))
x = layers.Dense(32, activation=activation)(input_tensor)
x = layers.Dense(32, activation=activation)(x)
output_tensor = layers.Dense(1, activation='softmax')(x)
model = Model(input_tensor, output_tensor)
model.compile(optimizer = opt,loss = "mean_absolute_error",metrics=['mse', "mape"])
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                288       
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 1,377
Trainable params: 1,377
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train,epochs=2,validation_data=(x_val,y_val))

Epoch 1/2
17/17 [==============================] - 0s 10ms/step - loss: 34.4839 - mse: 1483.8734 - mape: 96.1277 - val_loss: 34.7658 - val_mse: 1471.9219 - val_mape: 96.0676
Epoch 2/2
17/17 [==============================] - 0s 4ms/step - loss: 34.4839 - mse: 1483.8733 - mape: 96.1278 - val_loss: 34.7658 - val_mse: 1471.9219 - val_mape: 96.0676


### Hyperperameter optimizatoin (Functional Ali)

In [ ]:
n_cols = x_train.shape[1]
input_shape = (n_cols, )# input shape

#from keras.models import Sequential
from keras import regularizers #for l2 regularization
from keras.layers import Dense, Dropout
#from keras.optimizers import Adam

def create_model(learning_rate = 0.01, activation = 'relu'):
 # Create an Adam optimizer with the given learning rate
  opt = Adam(lr=learning_rate)
  # Create your binary classification model 
  input_tensor = Input(shape=(8,))
  x = layers.Dense(128, activation=activation,activity_regularizer = regularizers.l2(1e-5))(input_tensor)
  x = layers.Dropout(0.50)(x)
  x = layers.Dense(128, activation=activation,activity_regularizer = regularizers.l2(1e-5))(x)
  x = layers.Dropout(0.50)(x)
  output_tensor = layers.Dense(1, activation='softmax')(x)
  model = Model(input_tensor, output_tensor)
  # Compile the model
  model.compile(optimizer = opt,loss = "mean_absolute_error",metrics=['mse', "mape"])
  return model

In [ ]:
# onlu checking if the model is created corectly
learning_rate = 0.01
activation = 'relu'
#epochs = 50
#batch_size = 32 

model = create_model(learning_rate, activation)
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1152      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 17,793
Trainable params: 17,793
Non-trainable params: 0
__________________________________________________

In [ ]:
# Create a KerasClassifier object
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
#from sklearn.model_selection import cross_val_score

model = KerasRegressor(build_fn = create_model,verbose = 0)# Define the hyperparameter space
params = {'activation': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [5, 10], # [50, 100]
          'learning_rate': [0.01, 0.001, 0.0001]}# Create a randomize search cv object 

random_search = RandomizedSearchCV(model,param_distributions = params,cv = KFold(10))

In [ ]:
random_search_results = random_search.fit(x_train, y_train)

In [ ]:
print("Best Score: ",random_search_results.best_score_,
 "and Best Params: ",random_search_results.best_params_)

Best Score:  -34.48199653625488 and Best Params:  {'learning_rate': 0.01, 'epochs': 10, 'batch_size': 16, 'activation': 'tanh'}


### Predicting in Functional APi
https://stackoverflow.com/questions/38971293/get-class-labels-from-keras-functional-model/45176824#45176824

In [ ]:
"""#random_search.predict(x_test)
import keras

y_prob = random_search.predict(x_test) 
y_classes = keras.np_utils.probas_to_classes(y_proba)
#y_classes = y_prob.argmax(axis=-1)
y_classes

#a = random_search.predict(x_test)
#np.argmax(a)

#y_prob = model.predict(x) 
#y_classes = y_prob.argmax(axis=-1)"""

AttributeError: ignored

In [ ]:
"""# we can use the best model above directly of ML

prediction = random_search.predict(x_test)
#prediction

import numpy as np
predict = 41
print("Predicted Value :",prediction[predict])
print("Actual Value :",prediction[predict])"""

Predicted Value : 1.0
Actual Value : 1.0


In [ ]:
"""# we can use the best model above directly of ML

import numpy as np

prediction = random_search.np.argmax(predict(x_test))
prediction

#import numpy as np
#predict = 41
a = np.argmax(prediction[predict])
#print("Predicted Value :",prediction[predict])
#print("Actual Value :",prediction[predict])
a"""

AttributeError: ignored

In [ ]:
"""
predict = 7
a = np.argmax(final_preds[predict])
print("Predicted value: ",a)
print("Actual value: ",np.argmax(y_test[predict]))"""

In [ ]:
####### Prediction codes might be wrong, i will update them later inshallah ##########

### 10 Hyperparameter optimization frameworks.
https://towardsdatascience.com/10-hyperparameter-optimization-frameworks-8bc87bc8b7e3